# OpenFermion Fermionic Quantum Emulator 
## Phase I Design Proposal

This document outlines the functionality of the API for the OpenFermion Fermionic Quantum Emulator as defined in the design proposal.

 The code is covered and tested with pytest-cov at 99%. Toggle the print option to see output.

In [1]:
fqeprint = True

### 2.A.2 API for data conversion between OpenFermion's backend and OpenFermion-FQE
The API supports data conversion between native Cirq wavefunctions and the FQE wavefunction for arbitrary particle number and spin configuration.  A linequbit wavefunction result is a normalized numpy array.

In [2]:
import fqe
import numpy
import cirq

nqubit = 4

cirq_wfn = numpy.random.rand(2**nqubit).astype(numpy.complex128) + numpy.random.rand(2**nqubit).astype(numpy.complex128)*1.j
norm = numpy.sqrt(numpy.vdot(cirq_wfn, cirq_wfn))
numpy.divide(cirq_wfn, norm, out=cirq_wfn)
wfn_from_cirq = fqe.from_cirq(cirq_wfn, 0.0001)

if fqeprint:
    print('Inital wavefunction is \n {} \n with norm {}'.format(cirq_wfn, numpy.vdot(cirq_wfn, cirq_wfn)))
    wfn_from_cirq.print_wfn()

Inital wavefunction is 
 [0.13021848+0.10604389j 0.26332561+0.24265186j 0.29492744+0.28493664j
 0.16027682+0.11131328j 0.28246029+0.23136724j 0.13826045+0.07071791j
 0.0136616 +0.20124395j 0.04977281+0.15287608j 0.01397691+0.20703857j
 0.23519529+0.0175475j  0.07599475+0.08382844j 0.00162126+0.22461263j
 0.20667498+0.05097401j 0.24164648+0.00695315j 0.22218586+0.07588733j
 0.29277687+0.08259798j] 
 with norm (1+0j)
Sector N = 0 : S_z = 0
a'00'b'00' (0.1302184831092495+0.10604388636032029j)
Sector N = 1 : S_z = -1
a'00'b'01' (0.2824602875439038+0.23136723950216423j)
a'00'b'10' (0.26332561335087656+0.24265185521834995j)
Sector N = 1 : S_z = 1
a'01'b'00' (0.013976909592975447+0.20703856990343883j)
a'10'b'00' (0.29492743938955734+0.28493664175385763j)
Sector N = 2 : S_z = -2
a'00'b'11' (0.13826045473912837+0.07071791419269755j)
Sector N = 2 : S_z = 0
a'01'b'01' (0.2066749770899652+0.050974011826395334j)
a'01'b'10' (0.23519529197839167+0.017547500940564145j)
a'10'b'01' (-0.01366160068946521

### 2.A.3 Initialization of wave functions
FQE wave functions can be initialized by calling the constructor directly. fqe.get_wavefunction and get_wavefunction_mutltiple are utility wrappers that provide wrappers for common initialization motifs. e.g. get_spin_nonconserving_wavefunction will build a wavefunction that permits spin breaking Operators to be applied while get_wavefunction_multiple will return a list of initialize wavefunctions

In [3]:
wfFqe = fqe.Wavefunction([[4, -2, 10]], broken=None)
wfFqe = fqe.get_wavefunction(4, -2, 10)

In [4]:
n = 3
m = -2
wfFQE = fqe.Wavefunction([[n, m, 10], [n+1, m+2, 10]], broken=None)
wfFQE = fqe.get_wavefunction_multiple([[4, 0, 10], [5, -5, 10]])

In [5]:
wfFQE = fqe.get_number_conserving_wavefunction(2, 4)
wfFQE.set_wfn(strategy='random')
if fqeprint:
    wfFQE.print_wfn()

Sector N = 2 : S_z = -2
a'0000'b'0011' (0.22997775343026733+0.08615295784881312j)
a'0000'b'0101' (0.05516335203892644-0.46076275890197155j)
a'0000'b'1001' (0.2589715413729897-0.24055433543776833j)
a'0000'b'0110' (-0.08893603209627304-0.2056697723402523j)
a'0000'b'1010' (-0.5762638511993537-0.13426846293402575j)
a'0000'b'1100' (-0.046221179772583165-0.4437970493210447j)
Sector N = 2 : S_z = 0
a'0001'b'0001' (-0.2009463741491089+0.30702328278202784j)
a'0001'b'0010' (0.08055742138713175-0.27625908617663925j)
a'0001'b'0100' (-0.041261242667836505+0.343476586718014j)
a'0001'b'1000' (-0.08156556195302686-0.003425730152197814j)
a'0010'b'0001' (-0.04267181440534871+0.388798782448254j)
a'0010'b'0010' (0.1100960127727151-0.3188534929164057j)
a'0010'b'0100' (-0.2012398314866085-0.036134076630434914j)
a'0010'b'1000' (-0.04528159948796421-0.06986238599874375j)
a'0100'b'0001' (-0.0227584011700199-0.18576559249370736j)
a'0100'b'0010' (0.07097346928252087-0.1346627873998385j)
a'0100'b'0100' (-0.240445

### 2.A.4 Element access

Element access is modified from the original propsal to add more convient getter and setter functionalities.  The orginally proposed methods are availible with an addition function to add a value to an existing element.

In [6]:
wfFQE[(1, 8)] = .14159265359j
wfFQE[(1, 8)] += 3.j
imaginary_pi = wfFQE[(1, 8)]
if fqeprint:
    print(-imaginary_pi*1.j)

(3.14159265359-0j)


The new components not originally in the proposal allow access of entire vectors for getting and setting methods.

In [7]:
interesting_states = wfn_from_cirq.get_coeff((2, 0))
if fqeprint:
    print(interesting_states)

[[ 0.20667498+0.05097401j  0.23519529+0.0175475j ]
 [-0.0136616 -0.20124395j  0.16027682+0.11131328j]]


In [8]:
interesting_states[:, 1] += 1. + .1j
interesting_states *= .7 - .3j
if fqeprint:
    print(interesting_states)

[[ 0.15996469-0.02632068j  0.89990095-0.28827534j]
 [-0.0699363 -0.13677228j  0.87558776-0.20016375j]]


In [9]:
data = {}
data[(2, 0)] = interesting_states
wfn_from_cirq.set_wfn(strategy='from_data', raw_data=data)
if fqeprint:
    wfn_from_cirq.print_wfn()

Sector N = 0 : S_z = 0
a'00'b'00' (0.1302184831092495+0.10604388636032029j)
Sector N = 1 : S_z = -1
a'00'b'01' (0.2824602875439038+0.23136723950216423j)
a'00'b'10' (0.26332561335087656+0.24265185521834995j)
Sector N = 1 : S_z = 1
a'01'b'00' (0.013976909592975447+0.20703856990343883j)
a'10'b'00' (0.29492743938955734+0.28493664175385763j)
Sector N = 2 : S_z = -2
a'00'b'11' (0.13826045473912837+0.07071791419269755j)
Sector N = 2 : S_z = 0
a'01'b'01' (0.15996468751089424-0.026320684848512833j)
a'01'b'10' (0.8999009546670435-0.28827533693512264j)
a'10'b'01' (-0.06993630452078779-0.13677228254887208j)
a'10'b'10' (0.875587757741043-0.2001637491744398j)
Sector N = 2 : S_z = 2
a'11'b'00' (0.0759947496392153+0.08382844453325253j)
Sector N = 3 : S_z = -1
a'01'b'11' (0.2416464818447311+0.006953154640125172j)
a'10'b'11' (-0.04977281213093627-0.1528760837434253j)
Sector N = 3 : S_z = 1
a'11'b'01' (-0.222185860038206-0.07588732660218613j)
a'11'b'10' (0.0016212648925604868+0.22461262965671105j)
Sector

### 2.A.5 Printing/Saving

Printing is currently availible as alpha beta strings followed by the coefficient as well as orbital occupation representation.

In [10]:
print('String forrmatting')
wfn_from_cirq.print_wfn(fmt='str')
print('\n Occupation forrmatting')
wfn_from_cirq.print_wfn(fmt='occ')

String forrmatting
Sector N = 0 : S_z = 0
a'00'b'00' (0.1302184831092495+0.10604388636032029j)
Sector N = 1 : S_z = -1
a'00'b'01' (0.2824602875439038+0.23136723950216423j)
a'00'b'10' (0.26332561335087656+0.24265185521834995j)
Sector N = 1 : S_z = 1
a'01'b'00' (0.013976909592975447+0.20703856990343883j)
a'10'b'00' (0.29492743938955734+0.28493664175385763j)
Sector N = 2 : S_z = -2
a'00'b'11' (0.13826045473912837+0.07071791419269755j)
Sector N = 2 : S_z = 0
a'01'b'01' (0.15996468751089424-0.026320684848512833j)
a'01'b'10' (0.8999009546670435-0.28827533693512264j)
a'10'b'01' (-0.06993630452078779-0.13677228254887208j)
a'10'b'10' (0.875587757741043-0.2001637491744398j)
Sector N = 2 : S_z = 2
a'11'b'00' (0.0759947496392153+0.08382844453325253j)
Sector N = 3 : S_z = -1
a'01'b'11' (0.2416464818447311+0.006953154640125172j)
a'10'b'11' (-0.04977281213093627-0.1528760837434253j)
Sector N = 3 : S_z = 1
a'11'b'01' (-0.222185860038206-0.07588732660218613j)
a'11'b'10' (0.0016212648925604868+0.2246126

### 2.A.6 Fermionic algebra operations and their unitaries on the state

FermionOperators can be directly passed in to create a newwavefunction based on application of the operators.

In [11]:
from openfermion import FermionOperator
ops = FermionOperator('2^ 0', 1.2)
new_wfn = wfn_from_cirq.apply(ops)

AssertionError: 

In [12]:
new_wfn = fqe.apply(FermionOperator('0^ 0', 1.0), wfFQE)
if fqeprint:
    wfFQE.print_wfn()
    new_wfn.print_wfn()

Sector N = 2 : S_z = -2
a'0000'b'0011' (0.22997775343026733+0.08615295784881312j)
a'0000'b'0101' (0.05516335203892644-0.46076275890197155j)
a'0000'b'1001' (0.2589715413729897-0.24055433543776833j)
a'0000'b'0110' (-0.08893603209627304-0.2056697723402523j)
a'0000'b'1010' (-0.5762638511993537-0.13426846293402575j)
a'0000'b'1100' (-0.046221179772583165-0.4437970493210447j)
Sector N = 2 : S_z = 0
a'0001'b'0001' (-0.2009463741491089+0.30702328278202784j)
a'0001'b'0010' (0.08055742138713175-0.27625908617663925j)
a'0001'b'0100' (-0.041261242667836505+0.343476586718014j)
a'0001'b'1000' 3.14159265359j
a'0010'b'0001' (-0.04267181440534871+0.388798782448254j)
a'0010'b'0010' (0.1100960127727151-0.3188534929164057j)
a'0010'b'0100' (-0.2012398314866085-0.036134076630434914j)
a'0010'b'1000' (-0.04528159948796421-0.06986238599874375j)
a'0100'b'0001' (-0.0227584011700199-0.18576559249370736j)
a'0100'b'0010' (0.07097346928252087-0.1346627873998385j)
a'0100'b'0100' (-0.24044589692997007+0.1298869139786151

The fqe has vdot functionality to get an inner product and provides a way to do completely general expectation values.

In [13]:
ops = FermionOperator('2^ 2', 0.5)
opexpec = fqe.vdot(wfFQE, wfFQE.apply(ops))
if fqeprint:
    print(opexpec)

(0.47760569728890356+0j)


In [14]:
ops = FermionOperator('2^ 2', 0.5)
opexpec = fqe.vdot(wfFQE, fqe.apply(ops, wfFQE))
if fqeprint:
    print(opexpec)

(0.47760569728890356+0j)


We can also do general unitary application of operators.  One slight modification to the proposal is that the operator passed in should Hermetian and not anti-Hermitian as stated before.

In [15]:
ops = FermionOperator('2^ 0', 0.2 - .3j) + FermionOperator('0^ 2', 0.2 + .3j)
wfFQE.print_wfn()
wfFQE2 = wfFQE.apply_generated_unitary(0.001, 'taylor', ops)
wfFQE2.print_wfn()
ops = FermionOperator('2^ 0', 0.2 - .3j) + FermionOperator('0^ 2', 0.2 + .3j)
wfFQE2 = fqe.apply_generated_unitary(wfFQE, 0.001, 'taylor', ops)
wfFQE2.print_wfn()

Sector N = 2 : S_z = -2
a'0000'b'0011' (0.22997775343026733+0.08615295784881312j)
a'0000'b'0101' (0.05516335203892644-0.46076275890197155j)
a'0000'b'1001' (0.2589715413729897-0.24055433543776833j)
a'0000'b'0110' (-0.08893603209627304-0.2056697723402523j)
a'0000'b'1010' (-0.5762638511993537-0.13426846293402575j)
a'0000'b'1100' (-0.046221179772583165-0.4437970493210447j)
Sector N = 2 : S_z = 0
a'0001'b'0001' (-0.2009463741491089+0.30702328278202784j)
a'0001'b'0010' (0.08055742138713175-0.27625908617663925j)
a'0001'b'0100' (-0.041261242667836505+0.343476586718014j)
a'0001'b'1000' 3.14159265359j
a'0010'b'0001' (-0.04267181440534871+0.388798782448254j)
a'0010'b'0010' (0.1100960127727151-0.3188534929164057j)
a'0010'b'0100' (-0.2012398314866085-0.036134076630434914j)
a'0010'b'1000' (-0.04528159948796421-0.06986238599874375j)
a'0100'b'0001' (-0.0227584011700199-0.18576559249370736j)
a'0100'b'0010' (0.07097346928252087-0.1346627873998385j)
a'0100'b'0100' (-0.24044589692997007+0.1298869139786151

### 3.B.1 Hamiltonian data structure

Hamiltonians can be incorporated from OpenFermion objects or initialized from numpy arrays.

In [16]:
from openfermion import get_quadratic_hamiltonian
ops = FermionOperator((), 1.) + FermionOperator('1^ 1', 3.) + FermionOperator('1^ 2', 3. + 4.j) + FermionOperator('2^ 1', 3. - 4.j) + FermionOperator('3^ 4', 2. + 5.j) + FermionOperator('4^ 3', 2. - 5.j)
#hamOF = get_quadratic_hamiltonian(ops, -0.03)
hamFQE = fqe.get_hamiltonian_from_openfermion(ops, e_0=-0.03)
#hamFQE = fqe.get_quadratic_hamiltonian(ops, -0.03)

Symmetry is passed in by lists containing permutation operations, a phase and whether or not complex conjugation is used.  Below are examples of a Hermetian matrix and a real anti-symmetric matrix.

In [17]:
if False:
    e = 1.0
    t = numpy.random.rand(4, 4) + numpy.random.rand(4, 4)*1.j
    h = t + numpy.conjugate(t.T)
    symmh = [
        [[1, 2], 1.0, False],
        [[2, 1], 1.0, True]
    ]
    t = numpy.random.rand(4, 4, 4, 4)
    g = t - t.T
    symmg = [
        [[1, 2, 3, 4], 1.0, True],
        [[4, 3, 2, 1], -1.0, True]
    ]
    hamFQE = fqe.get_two_body_hamiltonian(-0.3, h, g, 0.0, symmh, symmg)

The FQE will also return PolynomialTensor objects for parsing by OpenFermion objects.

In [18]:
#hamOF = fqe.hamiltonian_to_openfermion(hamFQE)

### Saving and Loading Wavefunctions

Saving or loading a wavefunction is accomplished by passing a file name to the API and optionally a path to that filename.  If no path is given, the current working directory will be used.  If a path is used, the file will be searched for as

path + '/' + filename

In [19]:
wfnFqe = fqe.Wavefunction([[2, 2, 2], [2, 0, 2], [2, -2, 2]], broken=None)
wfnFqe.set_wfn(strategy='random')
wfnFqe.print_wfn()

Sector N = 2 : S_z = -2
a'00'b'11' (0.9830583946266548-0.18329264238933415j)
Sector N = 2 : S_z = 0
a'01'b'01' (0.5666188054469102-0.09258781162757845j)
a'01'b'10' (0.1986431337834743-0.3948732374798262j)
a'10'b'01' (0.3248834358204552+0.5372845502295152j)
a'10'b'10' (-0.2458007780800492-0.1426348515763947j)
Sector N = 2 : S_z = 2
a'11'b'00' (0.21268873797040513+0.9771200032445126j)


In [20]:
wfnFqe.save('random_wavefunction')
# wfnFqe.save('random_wavefunction', path='/home/google/wavefunctions')

Once it is saved you can return to the wavefunction by creating new object and then 'read' the data into it with the same syntax.

In [21]:
progress = fqe.Wavefunction([], broken=None)
progress.read('random_wavefunction')
# progress.read('random_wavefunction', path='/home/google/wavefunctions')
progress.print_wfn()

Sector N = 2 : S_z = -2
a'00'b'11' (0.9830583946266548-0.18329264238933415j)
Sector N = 2 : S_z = 0
a'01'b'01' (0.5666188054469102-0.09258781162757845j)
a'01'b'10' (0.1986431337834743-0.3948732374798262j)
a'10'b'01' (0.3248834358204552+0.5372845502295152j)
a'10'b'10' (-0.2458007780800492-0.1426348515763947j)
Sector N = 2 : S_z = 2
a'11'b'00' (0.21268873797040513+0.9771200032445126j)
